In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from textblob import TextBlob
import time

In [28]:
# File Path
fakenews_path = ("C:\\Users\\gyiko\\OneDrive - personalmicrosoftsoftware.uci.edu\\STATS\\STATS 170AB\\Project\\datasets\\factcheck\\data.json")

# Fact Check Dataset

In [29]:
fact_check = pd.read_json(fakenews_path)

In [30]:
texts = []
date = []
author_type = []
author = []
url = []
rating_type = []
rating = []
dataFeedElement = []

count=0
for item in fact_check["dataFeedElement"]:
    dataFeedElement.append(item)
    try:
        t = item['item'][0]['claimReviewed']
        texts.append(t)
        #language.append(TextBlob(t).detect_language()) # Detect the language of the text
    except:
        texts.append(None)
        #language.append(None)
    try:
        date.append(item['item'][0]['datePublished'])
    except:
        date.append(None)
    try:
        author_type.append(item['item'][0]['author']['@type'])
    except:
        author_type.append(None)
    try:
        author.append(item['item'][0]['author']['name'])
    except:
        author.append(None)
    try:
        url.append(item['item'][0]['author']['url'])
    except:
        url.append(None)
    try:
        rating_type.append(item['item'][0]['reviewRating']['@type'])
    except:
        rating_type.append(None)
    try:
        rating.append(item['item'][0]['reviewRating']['alternateName'])
    except:
        rating.append(None)

In [31]:
FactCheckData=pd.DataFrame({'text':texts,'date':date,'author_type':author_type,'author':author,'url':url,
              'rating_type':rating_type,'rating':rating,'dataFeedElement':dataFeedElement})

In [35]:
FactCheckData.head()

,text,date,author_type,author,url,rating_type,rating,dataFeedElement
0,"Assessora do secretário de Saúde do DF: ""ele v...",2021-03-12,Organization,Boatos.org,https://www.boatos.org/,Rating,Falso,"{'@type': 'DataFeedItem', 'dateCreated': '2021..."
1,George Soros em uma entrevista ontem para o Ne...,2021-03-12,Organization,Boatos.org,https://www.boatos.org/,Rating,Falso,"{'@type': 'DataFeedItem', 'dateCreated': '2021..."
2,"Heiko Maas hat gesagt: «Die Ehe, wenn auch als...",2021-03-12,Organization,dpa-factchecking,https://dpa-factchecking.com/,Rating,"Es gibt keinerlei Hinweise darauf, dass das Zi...","{'@type': 'DataFeedItem', 'dateCreated': '2021..."
3,Bereits positiv Geteste Corona-Infizierte werd...,2021-03-12,Organization,BR24 Faktenfuchs,https://www.br.de/,Rating,Falschbehauptung. Die Gesundheitsämter gleiche...,"{'@type': 'DataFeedItem', 'dateCreated': '2021..."
4,Artikel in The Lancet haalt PCR-testen onderuit,2021-03-12,Organization,dpa,https://dpa-factchecking.com/,Rating,Het artikel in The Lancet gaat over een vergel...,"{'@type': 'DataFeedItem', 'dateCreated': '2021..."


## Extract Content and Remove Null

In [33]:
FactCheckData_clean = FactCheckData.dropna().reset_index(drop=True)

In [40]:
FactCheckData_clean.head()

,text,date,author_type,author,url,rating_type,rating,dataFeedElement
0,"Assessora do secretário de Saúde do DF: ""ele v...",2021-03-12,Organization,Boatos.org,https://www.boatos.org/,Rating,Falso,"{'@type': 'DataFeedItem', 'dateCreated': '2021..."
1,George Soros em uma entrevista ontem para o Ne...,2021-03-12,Organization,Boatos.org,https://www.boatos.org/,Rating,Falso,"{'@type': 'DataFeedItem', 'dateCreated': '2021..."
2,"Heiko Maas hat gesagt: «Die Ehe, wenn auch als...",2021-03-12,Organization,dpa-factchecking,https://dpa-factchecking.com/,Rating,"Es gibt keinerlei Hinweise darauf, dass das Zi...","{'@type': 'DataFeedItem', 'dateCreated': '2021..."
3,Bereits positiv Geteste Corona-Infizierte werd...,2021-03-12,Organization,BR24 Faktenfuchs,https://www.br.de/,Rating,Falschbehauptung. Die Gesundheitsämter gleiche...,"{'@type': 'DataFeedItem', 'dateCreated': '2021..."
4,Artikel in The Lancet haalt PCR-testen onderuit,2021-03-12,Organization,dpa,https://dpa-factchecking.com/,Rating,Het artikel in The Lancet gaat over een vergel...,"{'@type': 'DataFeedItem', 'dateCreated': '2021..."


## Detect Language

In [ ]:
# TextBlob (already done to index 13513)

lan = []

count = 0
for text in FactCheckData_clean['text'][13514:20000]:
    print(count)
    if(len(text)>=3):
        lan.append(TextBlob(text).detect_language())
        time.sleep(2) # wait for 2 sec before next query in case of besing stuck by rate limit
    else:
        lan.append(None)
    count += 1

0


In [26]:
FactCheckData_clean['text']

0        Assessora do secretário de Saúde do DF: "ele v...
1        George Soros em uma entrevista ontem para o Ne...
2        Heiko Maas hat gesagt: «Die Ehe, wenn auch als...
3        Bereits positiv Geteste Corona-Infizierte werd...
4          Artikel in The Lancet haalt PCR-testen onderuit
                               ...                        
13509    “My name itself is a problem, Vappichi” : IIT ...
13510    Rezo muito para que esse óleo no nordeste não ...
13511    Claim cops manhandling woman over traffic chal...
13512    Claim Bhagavad Gita made compulsory for Dutch ...
13513    About 2% of pregnancy terminations take place ...
Name: text, Length: 13514, dtype: object

In [8]:
12978+536

13514

In [13]:
len(lan)

0

## Create Dataframe with Language and Save to JSON

In [9]:
FactCheckData_clean_mini = FactCheckData_clean[12978:13514]

In [10]:
len(FactCheckData_clean_mini)

536

In [11]:
FactCheckData_clean_mini = FactCheckData_clean_mini.assign(language=lan)

In [45]:
FactCheckData_clean_mini

,text,date,author_type,author,url,rating_type,rating,dataFeedElement,language
4886,"Im Internet kursiert das Gerücht, Berge um die...",2020-07-28,Organization,Deutsche Presse-Agentur,https://dpa-factchecking.com/,Rating,Unweit der bosnischen Stadt Visoko sind pyrami...,"{'@type': 'DataFeedItem', 'dateCreated': '2020...",de
4887,100,2020-08-22,Organization,مستقبل الايفون والايباد,https://www.ilamsat.net/,Rating,5,"{'@type': 'DataFeedItem', 'dateCreated': '2020...",en
4888,100,2020-08-21,Organization,مستقبل الايفون والايباد,https://www.ilamsat.net/,Rating,5,"{'@type': 'DataFeedItem', 'dateModified': '202...",en
4889,100,2020-08-22,Organization,مستقبل الايفون والايباد,https://www.ilamsat.net/,Rating,5,"{'@type': 'DataFeedItem', 'dateCreated': '2020...",en
4890,"Covid tem uma vibração de 5,5 hz e morre acima...",2020-08-21,Organization,Comprova,https://projetocomprova.com.br/,Rating,FALSO: Não há nenhuma comprovação científica d...,"{'@type': 'DataFeedItem', 'dateCreated': '2020...",pt
...,...,...,...,...,...,...,...,...,...
7803,சிகிச்சை அளித்த மருத்துவரையே கரம் பிடித்த கொரோ...,2020-06-01,Organization,Youturn,https://youturn.in/,Rating,Fake,"{'@type': 'DataFeedItem', 'dateCreated': '2020...",ta
7804,സാധനം വാങ്ങി പോവാൻ സമ്മതിക്കില്ല കാക്കകൾ\nസൌദി...,2020-05-30,Organization,Fact Crescendo,https://www.malayalam.factcrescendo.com/,Rating,FALSE,"{'@type': 'DataFeedItem', 'dateCreated': '2020...",ml
7805,ఫోటోలు తాజాగా ఉత్తరాఖండ్ లోని అడవులలో చెలరేగిన...,2020-05-31,Organization,FACTLY,https://factly.in/,Rating,FALSE,"{'@type': 'DataFeedItem', 'dateCreated': '2020...",te
7806,Image of a Shivalinga unearthed during the ex...,2020-05-31,Organization,FACTLY,https://factly.in/,Rating,FALSE,"{'@type': 'DataFeedItem', 'dateCreated': '2020...",en


## Import Previous Dataframe with Language

In [7]:
mini_file_path = "C:\\Users\\gyiko\\OneDrive - personalmicrosoftsoftware.uci.edu\\STATS\\STATS 170AB\\Project\\datasets\\factcheck\\FactCheckData_clean_mini.json"

In [13]:
test = pd.read_json(mini_file_path)

In [12]:
len(test)

13514

In [14]:
test.head()

,text,date,author_type,author,url,rating_type,rating,dataFeedElement,language
0,"Assessora do secretário de Saúde do DF: ""ele v...",2021-03-12,Organization,Boatos.org,https://www.boatos.org/,Rating,Falso,"{'@type': 'DataFeedItem', 'dateCreated': '2021...",pt
1,George Soros em uma entrevista ontem para o Ne...,2021-03-12,Organization,Boatos.org,https://www.boatos.org/,Rating,Falso,"{'@type': 'DataFeedItem', 'dateCreated': '2021...",pt
2,"Heiko Maas hat gesagt: «Die Ehe, wenn auch als...",2021-03-12,Organization,dpa-factchecking,https://dpa-factchecking.com/,Rating,"Es gibt keinerlei Hinweise darauf, dass das Zi...","{'@type': 'DataFeedItem', 'dateCreated': '2021...",de
3,Bereits positiv Geteste Corona-Infizierte werd...,2021-03-12,Organization,BR24 Faktenfuchs,https://www.br.de/,Rating,Falschbehauptung. Die Gesundheitsämter gleiche...,"{'@type': 'DataFeedItem', 'dateCreated': '2021...",de
4,Artikel in The Lancet haalt PCR-testen onderuit,2021-03-12,Organization,dpa,https://dpa-factchecking.com/,Rating,Het artikel in The Lancet gaat over een vergel...,"{'@type': 'DataFeedItem', 'dateCreated': '2021...",nl


In [9]:
# Show English Content
test[test['language']=='en'].reset_index(drop=True).head()

,text,date,author_type,author,url,rating_type,rating,dataFeedElement,language
0,Valenzuela City is part of Bulacan province.,2021-03-12,Organization,Vera Files,https://verafiles.org/,Rating,False,"{'@type': 'DataFeedItem', 'dateCreated': '2021...",en
1,Returning overseas Filipino workers (OFWs) and...,2021-03-12,Organization,Vera Files,https://verafiles.org/,Rating,Misleading,"{'@type': 'DataFeedItem', 'dateCreated': '2021...",en
2,A photo shows a vehicle driving through war-to...,2021-03-12,Organization,Vera Files,https://verafiles.org/,Rating,False,"{'@type': 'DataFeedItem', 'dateCreated': '2021...",en
3,Photo of the truck carrying an anti-Congress m...,2021-03-11,Organization,FACTLY,https://factly.in/,Rating,FALSE,"{'@type': 'DataFeedItem', 'dateCreated': '2021...",en
4,Video shows Rahul Gandhi saying that Mahatma G...,2021-03-11,Organization,FACTLY,https://factly.in/,Rating,MISLEADING,"{'@type': 'DataFeedItem', 'dateCreated': '2021...",en


## Export to JSON File

In [15]:
new_FactCheckData_clean_mini = pd.concat([test,FactCheckData_clean_mini])

In [17]:
new_FactCheckData_clean_mini.to_json(".\\datasets\\factcheck\\FactCheckData_clean_mini.json")